In [ ]:
pip install mne

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 19.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import h5py
import os
import glob
import mne
import numpy as np
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

In [ ]:
# Specify the channel types
ch_types = ['eeg'] * 19
ch_names = ['EEG1', 'EEG2', 'EEG3', 'EEG4','EEG5','EEG6','EEG7','EEG8','EEG9','EEG10','EEG11','EEG12','EEG13','EEG14','EEG15','EEG16','EEG17','EEG18','EEG19']

# Specify the output directory for .fif files
output_directory_path = '/content/drive/MyDrive/eeg_signals/ADHD_fif'

# Specify the directory containing your .mat EEG files
directory_path1 = '/content/drive/MyDrive/eeg_signals/ADHD_part1'
directory_path2 = '/content/drive/MyDrive/eeg_signals/ADHD_part2'

# Get a list of all .mat files in the directory
mat_files1 = glob.glob(os.path.join(directory_path1, '*.mat'))
mat_files2 = glob.glob(os.path.join(directory_path2, '*.mat'))
mat_files=mat_files1 + mat_files2

# Loop through each file
for mat_file in mat_files:
    mat_contents = sio.loadmat(mat_file)
    electrode_data = np.array(mat_contents[os.path.splitext(os.path.basename(mat_file))[0]])
    data = [i for i in electrode_data.T]  # Transpose the data


    # Create info and RawArray
    raw_info = mne.create_info(ch_names=ch_names, sfreq=128, ch_types=ch_types)
    raw = mne.io.RawArray(data, info=raw_info)

    # Save the RawArray to a .fif file in the output directory
    output_filename = os.path.splitext(os.path.basename(mat_file))[0] + '.fif'
    output_filepath = os.path.join(output_directory_path, output_filename)
    raw.save(output_filepath, overwrite=True)
    print(f"Saved {output_filepath}")



print("Processing complete!")

In [ ]:
# Specify the output directory for .fif files
output_directory_path = '/content/drive/MyDrive/eeg_signals/Control_fif'

# Specify the directory containing your .mat EEG files
directory_path1 = '/content/drive/MyDrive/eeg_signals/Control_part1'
directory_path2 = '/content/drive/MyDrive/eeg_signals/Control_part2'

# Get a list of all .mat files in the directory
mat_files1 = glob.glob(os.path.join(directory_path1, '*.mat'))
mat_files2 = glob.glob(os.path.join(directory_path2, '*.mat'))
mat_files=mat_files1 + mat_files2

# Loop through each file
for mat_file in mat_files:
    mat_contents = sio.loadmat(mat_file)
    electrode_data = np.array(mat_contents[os.path.splitext(os.path.basename(mat_file))[0]])
    data = [i for i in electrode_data.T]  # Transpose the data


    # Create info and RawArray
    raw_info = mne.create_info(ch_names=ch_names, sfreq=128, ch_types=ch_types)
    raw = mne.io.RawArray(data, info=raw_info)

    # Save the RawArray to a .fif file in the output directory
    output_filename = os.path.splitext(os.path.basename(mat_file))[0] + '.fif'
    output_filepath = os.path.join(output_directory_path, output_filename)
    raw.save(output_filepath, overwrite=True)
    print(f"Saved {output_filepath}")



print("Processing complete!")

In [ ]:
import os

folder_path = '/content/drive/MyDrive/eeg_signals/ADHD_fif'
for filename in os.listdir(folder_path):
    if filename.endswith('.fif'):
        # read .fif file
        file_path = os.path.join(folder_path, filename)
        raw = mne.io.read_raw_fif(file_path ,preload=True)

        l_freq = 0.5  # Cutoff frequency in Hz (e.g., remove slow drifts below 1 Hz)
        # Apply Cutoff frequency filter
        filtered_data=raw.filter(l_freq=l_freq, h_freq=None, fir_design='firwin', phase='zero')  # None for no high-frequency cutoff

        fnotch = 50  # Notch filter frequency (e.g., remove 50 Hz power line noise)
        # Apply notch filter
        filteredd_data = filtered_data.notch_filter(freqs=fnotch, picks=None, fir_design='firwin', phase='zero')

        #applying zero padding on the file
        n_zeros = 2
        zero_padding = np.zeros(n_zeros)
        data = filteredd_data.get_data()
        # Create a new array to hold the padded data
        padded_data = np.zeros((data.shape[0], data.shape[1] + 2 * n_zeros))

        # Iterate over each channel and apply zero-padding
        for ch_idx in range(data.shape[0]):
            channel_data = data[ch_idx]
            channel_padded = np.concatenate((zero_padding, channel_data, zero_padding))
            padded_data[ch_idx] = channel_padded

        # Create a new info object
        info = raw.info

        # Create a new RawArray object with the modified data
        raw_padded = mne.io.RawArray(padded_data, info)

        #applying moving avg filter
        # Iterate over each channel
        for i in range(len(raw_padded.ch_names)):
            # Get the data for the current channel
            data = raw_padded.get_data()[i]

            # Calculate the rolling mean
            moving_avg = pd.Series(data).rolling(window=5, min_periods=1).mean()

            # Replace NaNs in the rolling mean with zeros
            moving_avg = moving_avg.fillna(0)

            # Update the data in the Raw object with the rolling mean
            raw_padded._data[i] = moving_avg.values


        # Apply spatial filtering using Common Average Reference (CAR)
        raw_car = raw_padded.set_eeg_reference('average', projection=True)

        # Apply the projection to actually re-reference the data
        raw_car.apply_proj()


        # Save the modified Raw object back to the same file
        raw_car.save(file_path, overwrite=True)


print("Processing complete!")

In [ ]:

folder_path = '/content/drive/MyDrive/eeg_signals/Control_fif'

for filename in os.listdir(folder_path):
    if filename.endswith('.fif'):
        # read .fif file
        file_path = os.path.join(folder_path, filename)
        raw = mne.io.read_raw_fif(file_path ,preload=True)

        l_freq = 0.5  # Cutoff frequency in Hz (e.g., remove slow drifts below 1 Hz)
        # Apply Cutoff frequency filter
        filtered_data=raw.filter(l_freq=l_freq, h_freq=None, fir_design='firwin', phase='zero')  # None for no high-frequency cutoff

        fnotch = 50  # Notch filter frequency (e.g., remove 50 Hz power line noise)
        # Apply notch filter
        filteredd_data = filtered_data.notch_filter(freqs=fnotch, picks=None, fir_design='firwin', phase='zero')

        #applying zero padding on the file
        n_zeros = 2
        zero_padding = np.zeros(n_zeros)
        data = filteredd_data.get_data()
        # Create a new array to hold the padded data
        padded_data = np.zeros((data.shape[0], data.shape[1] + 2 * n_zeros))

        # Iterate over each channel and apply zero-padding
        for ch_idx in range(data.shape[0]):
            channel_data = data[ch_idx]
            channel_padded = np.concatenate((zero_padding, channel_data, zero_padding))
            padded_data[ch_idx] = channel_padded

        # Create a new info object
        info = raw.info

        # Create a new RawArray object with the modified data
        raw_padded = mne.io.RawArray(padded_data, info)

        #applying moving avg filter
        # Iterate over each channel
        for i in range(len(raw_padded.ch_names)):
            # Get the data for the current channel
            data = raw_padded.get_data()[i]

            # Calculate the rolling mean
            moving_avg = pd.Series(data).rolling(window=5, min_periods=1).mean()

            # Replace NaNs in the rolling mean with zeros
            moving_avg = moving_avg.fillna(0)

            # Update the data in the Raw object with the rolling mean
            raw_padded._data[i] = moving_avg.values


        # Apply spatial filtering using Common Average Reference (CAR)
        raw_car = raw_padded.set_eeg_reference('average', projection=True)

        # Apply the projection to actually re-reference the data
        raw_car.apply_proj()


        # Save the modified Raw object back to the same file
        raw_car.save(file_path, overwrite=True)
print("Processing complete!")